##  Demo of openFDA output to model format conversion
GNU General Public License v3.0 - Robert Ietswaart

### Citation:
Robert Ietswaart<sup>\*,#</sup>, Seda Arat<sup>\*,#</sup>, Amanda X. Chen<sup>\*</sup>, 
Saman Farahmand<sup>\*</sup>, Bumjun Kim, William DuMouchel, 
Duncan Armstrong, Alexander Fekete, Jeffrey J. Sutherland<sup>#</sup>, Laszlo Urban<sup>#</sup>  
*Machine learning guided association of adverse drug reactions with in vitro target-based 
pharmacology* (2019), [BioRxiv; 750950](https://www.biorxiv.org/content/10.1101/750950v2).

In [8]:
import pandas as pd
import numpy as np
import copy

### Load the openFDA output

In [9]:
path='~/'
version='v1'
filename=version+'_compounds_FDA.csv'
pulled = pd.read_table(path+filename,sep='\t',index_col=0)
pulled=pulled.rename(columns={'#reports': 'N_reports','HGLT_per_report':'HLGT_per_report'})
pulled=pulled.drop('Unnamed: 0.1',axis=1)
pulled.loc[pulled['N_reports']=='error','N_reports']=-1
pulled['N_reports']=pulled['N_reports'].astype(dtype='int32')

### Merge of multiple compound names
In pulled, some (unique) Novartis compound ID numbers occur multiple times as there are multiple names (1 row per generic name). Additionally when OpenFDA is queried for a particular generic name, it might return the same ADR report as for another name of the same compound. Here you merge the result for each generic name into one row for each unique compound ID.

In [10]:
UNIQ_number=list(set(pulled['number']))
len(UNIQ_number)

2134

In [11]:
for i in range(len(UNIQ_number)):
    TEMP=list(copy.deepcopy(pulled['number'][pulled['number']==UNIQ_number[i]].index))
    print(TEMP)
   
    if len(TEMP)>1:
        if max(pulled.loc[TEMP,'N_reports'])>0:#there are reports
            print(max(pulled.loc[TEMP,'N_reports']))
            MAX_ID=pulled['N_reports'][TEMP].idxmax()#get ID of row with max number of reports, merge rest into that row
            print(MAX_ID)
            mrep_id=pulled['reports_id'][MAX_ID].split('|||')
            
            for idx in TEMP:
                if idx != MAX_ID:
                    if pulled.loc[idx,'N_reports'] > 0:
#                     (pulled['reports_id'][idx] != 'No matches found!') and \
#                         (pulled['reports_id'][idx] is not np.nan):
                        reps=pulled['reports_id'][idx].split('|||')
                        pts=pulled['PTs_per_report'][idx].split('|||')
                        hlgts=pulled['HLGT_per_report'][idx].split('|||')
                        socs=pulled['SOC_per_report'][idx].split('|||')

                        for j in range(len(reps)):
                            if reps[j] not in mrep_id:
                                mrep_id.append(reps[j])
                                pulled.loc[MAX_ID,'PTs_per_report']=pulled.loc[MAX_ID,'PTs_per_report']+'|||'+pts[j]
                                pulled.loc[MAX_ID,'HLGT_per_report']=pulled.loc[MAX_ID,'HLGT_per_report']+'|||'+hlgts[j]
                                pulled.loc[MAX_ID,'SOC_per_report']=pulled.loc[MAX_ID,'SOC_per_report']+'|||'+socs[j]
                    
                    pulled=pulled.drop(idx)#delete the row
                    print('del idx',idx)
            pulled.loc[MAX_ID,'N_reports']=len(mrep_id)
            pulled.loc[MAX_ID,'reports_id']='|||'.join(mrep_id)
            
        else:#none of them have any reports, keep only first row, drop rest
            pulled=pulled.drop(TEMP[1:])
            print('del idx',TEMP[1:])

#drop the column name as we have merged the results for different generic names
pulled=pulled.drop(['name'], axis=1)

[891]
[548]
[1090]
[1063]
[1641]
[2316]
[227, 228]
9
227
del idx 228
[1675]
[1467]
[281]
[39]
[1618]
[376]
[835, 836]
70
835
del idx 836
[1587]
[1979]
[1104]
[1315]
[2294]
[395]
[1753]
[415]
[1312]
[814]
[464]
[2160]
[524]
[1365]
[1221]
[2044]
[2153]
[1125]
[786]
[1203]
[2329]
[77]
[2083]
[26]
[1044]
[878]
[457]
[197]
[857]
[1717]
[1177]
[612]
[1660]
[67]
[1661]
[1374]
[1612]
[1263]
[735]
[1111]
[1595]
[169]
[508]
[1271]
[209]
[717]
[1321]
[778, 779]
197
778
del idx 779
[1062]
[306]
[2048, 2049]
del idx [2049]
[1542]
[764]
[1492]
[2281]
[2268]
[1257]
[1919]
[846]
[533]
[1787, 1788]
72
1787
del idx 1788
[1296]
[967]
[1333, 1334]
10
1333
del idx 1334
[2196]
[2088]
[768]
[2366]
[207]
[1996]
[1411]
[2375]
[1493]
[2062]
[1610]
[828]
[1397]
[1778]
[1983]
[191]
[1613]
[167]
[800]
[1143]
[622]
[2047]
[2022]
[1834]
[185]
[2029]
[895]
[1840]
[183]
[2101]
[2280]
[1619]
[1622]
[72]
[757]
[419]
[338]
[1410]
[89]
[316]
[1724]
[1956]
[1781]
[1522]
[2057]
[1206]
[2098]
[61]
[1687, 1688]
del idx [1688]

426
162
del idx 161
[1891]
[1914]
[1963]
[2009]
[1630]
[714]
[1325, 1326]
del idx [1326]
[537]
[1871]
[1929]
[808]
[1637, 1638]
del idx [1638]
[1710, 1711]
26
1711
del idx 1710
[1428]
[1461]
[1574]
[1033]
[2387, 2388]
del idx [2388]
[1858]
[2231]
[444]
[2230]
[1796]
[1733]
[2193]
[776]
[797]
[1470, 1471]
13
1470
del idx 1471
[1665]
[976, 977, 978]
3
977
del idx 976
del idx 978
[2002]
[824]
[397]
[1258]
[1386]
[2012]
[148]
[283]
[1446]
[1472]
[194]
[623]
[1760]
[894]
[2228]
[526]
[320]
[297]
[446]
[1462]
[1662]
[578]
[2389, 2390]
del idx [2390]
[1449]
[2100]
[367]
[278]
[1020]
[1212]
[2085]
[931]
[1437]
[2030]
[568]
[56]
[331]
[1867]
[2271]
[580]
[739]
[1056]
[2300]
[451]
[1616]
[107, 108]
del idx [108]
[1186]
[1904, 1905]
del idx [1905]
[974]
[11, 12]
2
11
del idx 12
[1961]
[629]
[328]
[718]
[1468, 1469]
del idx [1469]
[1382]
[458]
[677]
[2232]
[753]
[652]
[2385]
[1142]
[1714]
[2377]
[1924]
[1309]
[2053]
[81]
[1249]
[1450]
[1725]
[1968]
[1313]
[2072]
[86]
[1591]
[1149, 1150]
3
1149
del

del idx [247]
[1466]
[1689]
[172]
[1645, 1646]
1
1645
del idx 1646
[1293]
[927]
[2254]
[1529]
[806]
[2067, 2068]
del idx [2068]
[221]
[460]
[1025]
[1799]
[1848]
[1807]
[1268]
[699]
[1922, 1923]
5
1922
del idx 1923
[1682]
[2016]
[752]
[393]
[777]
[1993, 1994]
del idx [1994]
[1459]
[2243]
[43]
[404]
[820]
[1872]
[1260]
[249]
[120]
[2324]
[134]
[103]
[412]
[1040, 1041, 1042]
3
1041
del idx 1040
del idx 1042
[1754, 1755]
211
1754
del idx 1755
[1442]
[1579, 1580]
460
1579
del idx 1580
[1810]
[2392]
[443]
[83]
[289]
[1087]
[1986]
[489, 490]
del idx [490]
[492]
[1305]
[1285, 1286]
del idx [1286]
[1290, 1291]
del idx [1291]
[1430]
[1888, 1889]
del idx [1889]
[742]
[150]
[340]
[1690]
[1267]
[582]
[1726]
[151]
[855]
[736, 737]
2
737
del idx 736
[2173]
[1679]
[640]
[1512]
[1509]
[1356]
[1550]
[260]
[345, 346, 347, 348, 349, 350, 351, 352, 353]
1
345
del idx 346
del idx 347
del idx 348
del idx 349
del idx 350
del idx 351
del idx 352
del idx 353
[1882]
[1073]
[1572]
[222]
[971]
[2212, 2213, 2214, 2

In [12]:
pulled

,number,N_reports,reports_id,PTs_per_report,HLGT_per_report,SOC_per_report
0,cid1,0,NaN,NaN,NaN,NaN
6,cid2,-1,No matches found!,NaN,NaN,NaN
7,cid3,0,NaN,NaN,NaN,NaN
8,cid4,4,10006390|||10587031|||14502025|||9970018,second primary malignancy;acute myeloid leukae...,NaN;leukaemias|||leukaemias|||deliria (incl co...,"NaN;neoplasms benign, malignant and unspecifie..."
9,cid5,-1,No matches found!,NaN,NaN,NaN
11,cid6,4,12484303|||13239363|||14593341|||7982394-3,mouth ulceration;stevens-johnson syndrome;geni...,oral soft tissue conditions;epidermal and derm...,gastrointestinal disorders;skin and subcutaneo...
14,cid7,25,10058460|||10230879|||12212949|||12354214|||13...,hepatitis|||hepatitis|||angioedema;drug hypers...,hepatic and hepatobiliary disorders|||hepatic ...,hepatobiliary disorders|||hepatobiliary disord...
15,cid8,-1,No matches found!,NaN,NaN,NaN
17,cid9,-1,No matches found!,NaN,NaN,NaN
18,cid10,-1,No matches found!,NaN,NaN,NaN


In [13]:
len(pulled)#double check: should equal len(UNIQ_number)

2134

### Calculate occurrence proportions and their significance (1 or 0)

In [14]:
from scipy.stats import binom
from statsmodels.stats.multitest import fdrcorrection

def P_binomial_ocr(ocr_counts, N_reps,N_t):
    p1=1.0/N_t
    pval = 1-binom.cdf(ocr_counts-1, N_reps, p1)#pval=prob(x >= ocr)=1-prob(x<= ocr-1)=1-cdf(ocr-1)
    return pval

def discretize_ocr(dict_pval):
    alpha_FDR=0.01
    pvals=list(dict_pval.values())
    BOOL, qvals = fdrcorrection(pvals, alpha=alpha_FDR)
    keys = list(dict_pval.keys())
    for i in range(len(keys)):
        if BOOL[i]: 
            dict_pval[keys[i]]= 1
        else:
            dict_pval[keys[i]]= 0
    return dict_pval


In [15]:
def get_ocr(onerow,level,level_list):
    if onerow[level] is not np.nan:
        N_terms=len(level_list)
        N_r=onerow['N_reports']
        ocr_raw = dict.fromkeys(level_list, 0)
        ocr_discretized = dict.fromkeys(level_list, 0)
        reps=onerow[level].split('|||')
        for r in reps:
            terms=set(r.split(';'))
            for t in terms:
                try:
                    ocr_raw[t]+=1
                except KeyError:
                    pass
        for akey in ocr_raw.keys():
            ocr_discretized[akey] = P_binomial_ocr(ocr_raw[akey],N_r,N_terms)#generate pvals
            ocr_raw[akey] = float(ocr_raw[akey])/N_r
        ocr_discretized=discretize_ocr(ocr_discretized)
    else:
        ocr_raw = {}
        ocr_discretized = {}
    return ocr_raw, ocr_discretized

In [16]:
def calc_occurences(pulled,term_list,level):
    res_ocr_raw = {}
    res_ocr_disc = {}
    for asoc in term_list:
        res_ocr_raw[asoc] = []
        res_ocr_disc[asoc] = []

    for index, arow in pulled.iloc[:].iterrows():
        raw_tmp, disc_tmp  = get_ocr(arow,level,term_list)
        for asoc in res_ocr_raw.keys():
            try:
                res_ocr_raw[asoc].append(raw_tmp[asoc])
            except KeyError:
                res_ocr_raw[asoc].append(np.NaN)
            try:
                res_ocr_disc[asoc].append(disc_tmp[asoc])
            except KeyError:
                res_ocr_disc[asoc].append(np.NaN)
    
    return res_ocr_raw , res_ocr_disc

### function to output  results to csv

In [17]:
def ocr_to_csv(res_dict,pulled,path,filename):
    res_df = pd.DataFrame(res_dict)
    res_df.insert(loc=0,column='number', value=pulled['number'].tolist())
    res_df.insert(loc=1,column='N_reports', value=pulled['N_reports'].tolist())
    res_df.to_csv(path+filename)
    return res_df

### Write SOC level results to csv

In [18]:
soc_acc = []
for i in pulled['SOC_per_report'].tolist():
    if i is not np.nan:
        i=i.replace('|||',';')
        soc_acc.extend(i.split(';'))
soc_list = set(soc_acc)
soc_list.remove('NaN')
soc_list #System Organ Classes

{'blood and lymphatic system disorders',
 'cardiac disorders',
 'congenital, familial and genetic disorders',
 'ear and labyrinth disorders',
 'endocrine disorders',
 'eye disorders',
 'gastrointestinal disorders',
 'general disorders and administration site conditions',
 'hepatobiliary disorders',
 'immune system disorders',
 'infections and infestations',
 'injury, poisoning and procedural complications',
 'investigations',
 'metabolism and nutrition disorders',
 'musculoskeletal and connective tissue disorders',
 'neoplasms benign, malignant and unspecified (incl cysts and polyps)',
 'nervous system disorders',
 'pregnancy, puerperium and perinatal conditions',
 'psychiatric disorders',
 'renal and urinary disorders',
 'reproductive system and breast disorders',
 'respiratory, thoracic and mediastinal disorders',
 'skin and subcutaneous tissue disorders',
 'social circumstances',
 'surgical and medical procedures',
 'vascular disorders'}

In [19]:
res_ocr_raw,res_ocr_discretized=calc_occurences(pulled,soc_list,'SOC_per_report')

In [21]:
path='~/'
filename=version+'_compounds_FDA_model_format_SOC_ocr_prob_demo.csv'
res_df=ocr_to_csv(res_ocr_raw,pulled,path,filename)
res_df



,number,N_reports,hepatobiliary disorders,blood and lymphatic system disorders,nervous system disorders,social circumstances,psychiatric disorders,skin and subcutaneous tissue disorders,musculoskeletal and connective tissue disorders,surgical and medical procedures,...,"neoplasms benign, malignant and unspecified (incl cysts and polyps)",metabolism and nutrition disorders,reproductive system and breast disorders,"respiratory, thoracic and mediastinal disorders",eye disorders,"congenital, familial and genetic disorders","pregnancy, puerperium and perinatal conditions","injury, poisoning and procedural complications",cardiac disorders,investigations
0,cid1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cid2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cid3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cid4,4,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,...,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,cid5,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cid6,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,cid7,25,0.200000,0.040000,0.120000,0.000000,0.040000,0.200000,0.000000,0.040000,...,0.000000,0.200000,0.000000,0.040000,0.040000,0.000000,0.000000,0.080000,0.080000,0.080000
7,cid8,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cid9,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cid10,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
filename=version+'_compounds_FDA_model_format_SOC_ocr_bool_demo.csv'
res_df=ocr_to_csv(res_ocr_discretized,pulled,path,filename)
res_df

,number,N_reports,hepatobiliary disorders,blood and lymphatic system disorders,nervous system disorders,social circumstances,psychiatric disorders,skin and subcutaneous tissue disorders,musculoskeletal and connective tissue disorders,surgical and medical procedures,...,"neoplasms benign, malignant and unspecified (incl cysts and polyps)",metabolism and nutrition disorders,reproductive system and breast disorders,"respiratory, thoracic and mediastinal disorders",eye disorders,"congenital, familial and genetic disorders","pregnancy, puerperium and perinatal conditions","injury, poisoning and procedural complications",cardiac disorders,investigations
0,cid1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cid2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cid3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cid4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cid5,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cid6,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,cid7,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,cid8,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cid9,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cid10,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Write HLGT level results to csv
HLGT = Higher level Group Terms

In [23]:
hlgt_acc = []
for i in pulled['HLGT_per_report'].tolist():
    if i is not np.nan:
        i=i.replace('|||',';')
        hlgt_acc.extend(i.split(';'))
hlgt_list = set(hlgt_acc)
hlgt_list.remove('NaN')
hlgt_list

{'abdominal hernias and other abdominal wall conditions',
 'abortions and stillbirth',
 'acid-base disorders',
 'adjustment disorders (incl subtypes)',
 'administration site reactions',
 'adrenal gland disorders',
 'age related factors',
 'allergic conditions',
 'anaemias nonhaemolytic and marrow depression',
 'anal and rectal conditions nec',
 'ancillary infectious topics',
 'aneurysms and artery dissections',
 'angioedema and urticaria',
 'anterior eye structural change, deposit and degeneration',
 'anxiety disorders and symptoms',
 'appetite and general nutritional disorders',
 'arteriosclerosis, stenosis, vascular insufficiency and necrosis',
 'aural disorders nec',
 'autoimmune disorders',
 'bacterial infectious disorders',
 'benign neoplasms gastrointestinal',
 'bile duct disorders',
 'bladder and bladder neck disorders (excl calculi)',
 'blood and lymphatic system disorders congenital',
 'body temperature conditions',
 'bone and joint injuries',
 'bone and joint therapeutic proc

In [24]:
res_ocr_raw,res_ocr_discretized=calc_occurences(pulled,hlgt_list,'HLGT_per_report')

In [25]:
filename=version+'_compounds_FDA_model_format_HLGT_ocr_prob_demo.csv'
res_df=ocr_to_csv(res_ocr_raw,pulled,path,filename)
res_df

,number,N_reports,immune disorders nec,lymphomas non-hodgkin's t-cell,hepatobiliary investigations,middle ear disorders (excl congenital),glucose metabolism disorders (incl diabetes mellitus),reproductive neoplasms male malignant and unspecified,psychiatric and behavioural symptoms nec,thyroid gland disorders,...,hepatic and hepatobiliary disorders,iron and trace metal metabolism disorders,haemoglobinopathies,gastrointestinal investigations,embolism and thrombosis,viral infectious disorders,chromosomal abnormalities and abnormal gene carriers,salivary gland conditions,ovarian and fallopian tube disorders,"anterior eye structural change, deposit and degeneration"
0,cid1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cid2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cid3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cid4,4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,cid5,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cid6,4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,cid7,25,0.0,0.0,0.040000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.200000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,cid8,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cid9,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cid10,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
filename=version+'_compounds_FDA_model_format_HLGT_ocr_bool_demo.csv'
res_df=ocr_to_csv(res_ocr_discretized,pulled,path,filename)
res_df

,number,N_reports,immune disorders nec,lymphomas non-hodgkin's t-cell,hepatobiliary investigations,middle ear disorders (excl congenital),glucose metabolism disorders (incl diabetes mellitus),reproductive neoplasms male malignant and unspecified,psychiatric and behavioural symptoms nec,thyroid gland disorders,...,hepatic and hepatobiliary disorders,iron and trace metal metabolism disorders,haemoglobinopathies,gastrointestinal investigations,embolism and thrombosis,viral infectious disorders,chromosomal abnormalities and abnormal gene carriers,salivary gland conditions,ovarian and fallopian tube disorders,"anterior eye structural change, deposit and degeneration"
0,cid1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cid2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cid3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cid4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cid5,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cid6,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,cid7,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,cid8,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cid9,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cid10,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
